In [1]:
import tensorflow as tf
import torch

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import random
import time
import datetime
import csv
import torch.nn as nn

2024-05-10 09:07:03.082958: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-10 09:07:03.103347: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-10 09:07:03.419399: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
tokenizer = BertTokenizer.from_pretrained('kykim/bert-kor-base')
model = BertForSequenceClassification.from_pretrained('kykim/bert-kor-base', num_labels=318)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at kykim/bert-kor-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
model.load_state_dict(torch.load('훈련의 결과로 나온 모델 이름.pth'))

<All keys matched successfully>

In [4]:
# 입력 데이터 변환
def convert_input_data(sentences):
    sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]
    print(sentences)
    # BERT의 토크나이저로 문장을 토큰으로 분리
    tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
    print(tokenized_texts)
    # 입력 토큰의 최대 시퀀스 길이
    MAX_LEN = 64

    # 토큰을 숫자 인덱스로 변환
    input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
    
    # 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
    input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

    # 어텐션 마스크 초기화
    attention_masks = []

    # 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
    # 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
    for seq in input_ids:
        seq_mask = [float(i>0) for i in seq]
        attention_masks.append(seq_mask)

    # 데이터를 파이토치의 텐서로 변환
    inputs = torch.tensor(input_ids)
    masks = torch.tensor(attention_masks)

    return inputs, masks



# 문장 테스트
def test_sentences(sentences):

    # 평가모드로 변경
    model.eval()

    # 문장을 입력 데이터로 변환
    inputs, masks = convert_input_data(sentences)

    # 데이터를 GPU에 넣음
    b_input_ids = inputs
    b_input_mask = masks
            
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)

    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()

    return logits

In [5]:
logits = test_sentences([ "제주무1개",
    "청경채",
    "냉이 * 1봉",
    "쑥갓 2KG",
    "백설)행복한아식통통숙주380g",
    "팩느타리1팩",
    "표고버섯(국산) 1k",
    "참조기",
    "쭈꾸미",
    "새송이버섯(1인*4봉 한정)1봉",
    "알배기배추1통"])

['[CLS] 제주무1개 [SEP]', '[CLS] 청경채 [SEP]', '[CLS] 냉이 * 1봉 [SEP]', '[CLS] 쑥갓 2KG [SEP]', '[CLS] 백설)행복한아식통통숙주380g [SEP]', '[CLS] 팩느타리1팩 [SEP]', '[CLS] 표고버섯(국산) 1k [SEP]', '[CLS] 참조기 [SEP]', '[CLS] 쭈꾸미 [SEP]', '[CLS] 새송이버섯(1인*4봉 한정)1봉 [SEP]', '[CLS] 알배기배추1통 [SEP]']
[['[CLS]', '제주', '##무', '##1개', '[SEP]'], ['[CLS]', '청', '##경', '##채', '[SEP]'], ['[CLS]', '냉', '##이', '*', '1', '##봉', '[SEP]'], ['[CLS]', '쑥', '##갓', '2kg', '[SEP]'], ['[CLS]', '백설', ')', '행복한', '##아', '##식', '##통', '##통', '##숙', '##주', '##38', '##0', '##g', '[SEP]'], ['[CLS]', '팩', '##느', '##타리', '##1', '##팩', '[SEP]'], ['[CLS]', '표고버섯', '(', '국산', ')', '1', '##k', '[SEP]'], ['[CLS]', '참조', '##기', '[SEP]'], ['[CLS]', '쭈꾸미', '[SEP]'], ['[CLS]', '새', '##송이', '##버섯', '(', '1인', '*', '4', '##봉', '한정', ')', '1', '##봉', '[SEP]'], ['[CLS]', '알', '##배기', '##배추', '##1', '##통', '[SEP]']]


In [6]:
data = pd.read_csv("./result/label.csv")

for a in logits :
    #print(np.argmax(a))
    # 번호가 333에 해당하는 레이블을 찾습니다.
    label = data[data['number'] == np.argmax(a)]['label'].values[0]
    print("모델의 분류 결과:", label)

모델의 분류 결과: 채소/무
모델의 분류 결과: 채소/청경채
모델의 분류 결과: 채소/냉이
모델의 분류 결과: 채소/쑥갓
모델의 분류 결과: 채소/숙주나물
모델의 분류 결과: 채소/버섯
모델의 분류 결과: 채소/버섯
모델의 분류 결과: 생선/조기
모델의 분류 결과: 어패류/주꾸미
모델의 분류 결과: 채소/버섯
모델의 분류 결과: 채소/배추
